In [45]:
import numpy as np
from sympy import *
from scipy.linalg import eigh
    
# Tropical addition
# num - num
# n-vector - n-vector
# nm-matrix - nm-matrix
def tplus(a, b):
    
    if (len(np.shape(a)) == 2 and np.shape(a) == np.shape(b)):
        # perform term-wise matrix tropical addition
        
        n, m = np.shape(a)
        out = np.zeros(shape=(n,m))

        for i in range(n):
            for j in range(m):
                out[i][j] = tplus(a[i][j], b[i][j])
        return out
        
    elif (len(np.shape(a)) == len(np.shape(b)) == 1):
        # perform term-wise vector tropical addition
        
        n = np.shape(a)[0]
        out = np.zeros(n)

        for i in range(n):
            out[i] = tplus(a[i], b[i])
        return out
    
    # otherwise, we operate on scalars
    if (mode == 0):
        return min(a, b)
    else:
        return max(a, b)
    
    
# Normalize a tropical vector
# Ensure that the first term is the multiplicative identity
def normalize(v):
    if (len(np.shape(v))==2): # column vector
        return ttimes(texp(v[0][0],-1), v)
    
    else: # row vector
        return ttimes(texp(v[0],-1), v)


# Tropical multiplication
# num - num
# scalar - vector
# scalar - matrix
# nm-matrix - mx-matrix
# nm-matrix - m-vector
def ttimes(a, b):
    if (len(np.shape(a)) == 2): # a is a matrix
        
        # if b is a vector, ensure it is a column vector
        if(len(np.shape(b))==1):
            b = np.array([b]).T
        elif(np.shape(b)[0]==1):
            b = np.array(b).reshape(-1,1)

        assert np.shape(a)[1] == np.shape(b)[0]
        n, m = np.shape(a)
        
        m, x = np.shape(b)
        out = np.zeros(shape=(n,x))
        
        # perform matrix multiplication
        for i in range(n):
            for j in range(x):
                val = ttimes(a[i][0],b[0][j])
                
                for k in range(m):
                    val = tplus(val, ttimes(a[i][k],b[k][j]))
                out[i][j] = val
                
        # if we output a vector, normalize
        if (np.shape(out)[1] == 1):
            out = normalize(out)
        return out
    
    elif (len(np.shape(b)) == 1): # a is not a matrix, b is a vector
        # therefore a is a scalar
        assert(len(np.shape(a))==0)
        
        m = np.shape(b)[0]
        out = np.zeros(m)
        
        # perform scalar multiplication
        for i in range(m):
            out[i] = ttimes(b[i],a)
        return out
    
    elif (len(np.shape(b)) == 2): # a is not a matrix, b is a matrix
        # therefore a is a scalar
        assert(len(np.shape(a))==0)
        
        # perform scalar multiplication
        n, m = np.shape(b)
        out = np.zeros(shape=(n, m))
        
        for i in range(n):
            for j in range(m):
                out[i][j] = ttimes(b[i][j], a)
        return out
    
    # a and b are both scalars
    return (a + b) if (mode2==0) else (a * b)

# Tropical dot product
# n-vector - n-vector
def tdot(a, b):
    # ensure both vectors are flattened
    a = np.ndarray.flatten(a)
    b = np.ndarray.flatten(b)
    
    # perform a dot product
    val = ttimes(a[0], b[0])
    for i in range(len(a)):
        val = tplus(val, ttimes(a[i],b[i]))
    return val

# Tropical exponentiation
def texp(a, b):
    return (a * b) if (mode2==0) else (a**b)

# Calculate tropical eigenvalue of a square matrix
# TODO: ensure this works with mode2==1
def teigval(matrix):
    # ensure matrix is square
    n, m = np.shape(matrix)
    assert n == m
    
    ma = matrix # will store current exponent of matrix
    cur = matrix # will store current sum of matrix powers
    
    for i in range(1,n): # 1 to n-1, since the first power is already stored
        
        # compute matrix^(i+1)
        # this is the (i+1)-length optimal path matrix
        ma = ttimes(ma, matrix)
        
        # divide by path length, and combine with previous matrices
        cur = tplus(cur, (1/(i+1))*ma)
        
    # cur now stores the (1...n)-length optimal average-weight paths
    # find optimal value on diagonal (optimal cycle)
    val = cur[0][0]
    for i in range(n):
        val = tplus(val, cur[i][i])
        
    # optimal average-weight cycle weight is equal to tropical eigenvalue
    return val

# Calculate tropical eigenvectors of a square matrix
# TODO: ensure this works with mode2==1
def teigvec(matrix):
    # ensure matrix is square
    n, m = np.shape(matrix)
    assert n == m
    
    # subtract/divide out the eigenvalue
    eigval = teigval(matrix)
    matrix = ttimes(texp(eigval,-1), matrix)
    # eigval(matrix) is now the identity
    
    ma = matrix # will store current exponent of matrix
    cur = matrix # will store current sum of matrix powers
    
    for i in range(1,n): # 1 to n-1, since the first power is already stored
        
        # compute matrix^(i+1)
        # this is the (i+1)-length optimal path matrix
        ma = ttimes(ma, matrix)
        
        # combine with previous matrices
        cur = tplus(cur, ma)
    
    # cur now stores "C^+" matrix
    
    out = []
    cur = cur.transpose() # so we can index the columns
    
    # find columns where the diagonal entry is the identity
    for i in range(n):
        if (cur[i][i] == mode2):
            out.append(cur[i])
            
    # compute a normalized basis
    return basis(out)

# Calculate basis of given list of vectors
# TODO: ensure linear combination vectors are also removed
def basis(vectors):
    count = 1
    out = []
    
    # if input is empty, return empty list
    n = len(vectors)
    if (n==0):
        return out
    
    # add the first vector, normalized
    out.append(normalize(vectors[0]))
    
    for i in range(1,n): # iterate over the rest
        vectors[i] = normalize(vectors[i])
        matches = 0
        
        # see if it matches any already in the basis
        for j in range(0,len(out)):
            if ((out[j] == vectors[i]).all()):
                matches = 1
                
        # if no matches, append to output
        if (matches == 0):
            out.append(vectors[i])
            
    return out

# Compute transfer matrix (of a square matrix)
def transfer_matrix(A, h):
    n, m = np.shape(A)
    assert n == m
    out = np.zeros(shape=(n,m))

    for i in range(n):
        for j in range(m):
            out[i][j] = np.exp(A[i][j]*h)
    return out

# Compute transfer P-F eval/evecs
def transfer_ev(A, h):
    n,m = np.shape(A)
    E = transfer_matrix(A, h)
    val,vec = eigh(E)
    
    # DEBUG
    print()
    print(h)
    print(E)
    print(val)
    print(vec)
    
    # allow us to index into eigenvectors
    vec = vec.T
    
    # if smallest eval has largest absolute value, use it instead
    ind = n-1
    if (-1*val[0] > val[ind]):
        ind = 0
    
    # output vector
    ove = vec[ind]
    # ensure entries are positive
    if (ove[0] < 0):
        ove = -1*ove
        
    # compute normalization
    for i in range(len(ove)):
        ove[i] = (1/h)*np.log(ove[i])
        
    # output value
    ova = val[ind]
    
    # compute normalization
    ova = (1/h)*np.log(ova)
    
    return ova, ove            

In [46]:
# 0 : min
# 1 : max
mode = 1

# 0 : plus
# 1 : times
mode2 = 1

In [47]:
a = np.zeros(3)
b = np.zeros(3)
c = np.zeros(3)
a[0], a[1], a[2] = 1, 1, 1/5
b[0], b[1], b[2] = 1, 3, 1/2
c[0], c[1], c[2] = 5, 2, 1

In [48]:
m1 = np.zeros(shape=(3,3))
m2 = np.zeros(shape=(3,3))
m1[0], m1[1], m1[2] = a, b, c
m2[0], m2[1], m2[2] = a, b, c
m1 = np.log(m1)
print(m1)

v1 = np.zeros(shape=(3,1))
v2 = np.zeros(shape=(3,1))
v3 = np.zeros(shape=(3,1))
v1[0], v1[1], v1[2] = 0, -1, -1
v2[0], v2[1], v2[2] = 0, 2, 1
v3[0], v3[1], v3[2] = 0, 0, 1

[[ 0.          0.         -1.60943791]
 [ 0.          1.09861229 -0.69314718]
 [ 1.60943791  0.69314718  0.        ]]


In [49]:
print(teigval(m1))
print(teigval(m2))

1.0986122886681098
9.0


In [50]:
print(teigvec(m1))
print(teigvec(m2))

[array([nan, inf, inf])]
[]


/home/balazs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:127: RuntimeWarning: divide by zero encountered in double_scalars
/home/balazs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:110: RuntimeWarning: invalid value encountered in double_scalars


In [51]:
print(ttimes(m1, teigvec(m1)))

[[nan]
 [nan]
 [nan]]


/home/balazs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:127: RuntimeWarning: divide by zero encountered in double_scalars
/home/balazs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:110: RuntimeWarning: invalid value encountered in double_scalars


In [52]:
print(m1)
print(eigh(m1))
print()
print(transfer_ev(m1,1))
print(transfer_ev(m1,10))
print(transfer_ev(m1,100))
print(transfer_ev(m1,1000))
print(transfer_ev(m1,10000))

[[ 0.          0.         -1.60943791]
 [ 0.          1.09861229 -0.69314718]
 [ 1.60943791  0.69314718  0.        ]]
(array([-1.69763937,  0.87042105,  1.92583061]), array([[ 0.6771341 ,  0.50054819, -0.5393894 ],
       [ 0.17704966, -0.82229478, -0.54081856],
       [-0.71424284,  0.27070797, -0.64542572]]))


1
[[1.  1.  0.2]
 [1.  3.  0.5]
 [5.  2.  1. ]]
[-4.07542846  1.96777436  7.10765411]
[[ 0.68715499  0.40117128  0.60570589]
 [ 0.10604374 -0.88017402  0.46265367]
 [-0.71872995  0.25368346  0.64735768]]
(1.961172246778648, array([-0.50136075, -0.77077652, -0.43485631]))

10
[[1.000000e+00 1.000000e+00 1.024000e-07]
 [1.000000e+00 5.904900e+04 9.765625e-04]
 [9.765625e+06 1.024000e+03 1.000000e+00]]
[-9765624.05326025    59048.99914102  9765626.05411923]
[[ 7.07106777e-01  1.04862052e-04  7.07106777e-01]
 [ 7.36279196e-05 -9.99999995e-01  7.46694171e-05]
 [-7.07106781e-01  7.36449984e-07  7.07106781e-01]]
(1.609437923228281, array([-0.03465736, -0.950244  , -0.03465736]))

100

/home/balazs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:263: RuntimeWarning: divide by zero encountered in log


ValueError: array must not contain infs or NaNs